# 1. Load Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import sklearn
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
plt.style.use('seaborn')
sns.set(font_scale=2.5)
import missingno as msno

#ignore warnings
import warnings
warnings.filterwarnings('ignore') # 워닝 메세지를 생략해 줍니다. 차후 버전관리를 위해 필요한 정보라고 생각하시면 주석처리 하시면 됩니다.

%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
df_train = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
df_test = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_test.csv')
df_submit = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/sample_submission.csv')
df_train.head()

In [ ]:
df_train=df_train.dropna(axis=0)
df_test=df_test.dropna(axis=0)
df_train.isnull().sum()

In [ ]:
df_train.drop(['enrollee_id','city'], axis=1, inplace=True)
df_test.drop(['enrollee_id','city'], axis=1, inplace=True)

In [ ]:
keyword = 'last_new_job'
df_train[keyword] = df_train[keyword].replace({'never':0, '>4':5})
df_test[keyword] = df_test[keyword].replace({'never':0, '>4':5})

keyword = 'company_type'
df_train[keyword] = df_train[keyword].replace({'Pvt Ltd':0, 'Funded Startup':1, 'Public Sector':2, 'Early Stage Startup':3, 'NGO':4, 'Other':5})
df_test[keyword] = df_test[keyword].replace({'Pvt Ltd':0, 'Funded Startup':1, 'Public Sector':2, 'Early Stage Startup':3, 'NGO':4, 'Other':5})

keyword = 'company_size'
df_train[keyword] = df_train[keyword].replace({'<10':5, '10/49':25, '50-99':75, '100-500':250, '500-999':750, '1000-4999':2500, '5000-9999':7500, '10000+':10000})
df_test[keyword] = df_test[keyword].replace({'<10':5, '10/49':25, '50-99':75, '100-500':250, '500-999':750, '1000-4999':2500, '5000-9999':7500, '10000+':10000})

keyword = 'experience'
df_train[keyword] = df_train[keyword].replace({'>20':21, '<1':0})
df_test[keyword] = df_test[keyword].replace({'>20':21, '<1':0})

keyword = 'major_discipline'
df_train[keyword] = df_train[keyword].replace({'STEM':0, 'Humanities':1, 'Business Degree':2, 'Arts':3, 'No Major':4, 'Other':5})
df_test[keyword] = df_test[keyword].replace({'STEM':0, 'Humanities':1, 'Business Degree':2, 'Arts':3, 'No Major':4, 'Other':5})

keyword = 'education_level'
df_train[keyword] = df_train[keyword].replace({'Primary School':0, 'High School':1, 'Graduate':2, 'Masters':3, 'Phd':4})
df_test[keyword] = df_test[keyword].replace({'Primary School':0, 'High School':1, 'Graduate':2, 'Masters':3, 'Phd':4})

df_train['enrolled_university'] = df_train['enrolled_university'].replace({'no_enrollment':0, 'Part time course':1, 'Full time course':2})
df_test['enrolled_university'] = df_test['enrolled_university'].replace({'no_enrollment':0, 'Part time course':1, 'Full time course':2})

df_train['relevent_experience'] = df_train['relevent_experience'].replace({'Has relevent experience':1, 'No relevent experience' :0})
df_test['relevent_experience'] = df_test['relevent_experience'].replace({'Has relevent experience':1,'No relevent experience':0})

df_train['gender'] = df_train['gender'].replace({'Male':0, 'Female' :1})
df_test['gender'] = df_test['gender'].replace({'Male':0,'Female':1})

df_train.drop(df_train[df_train['gender']=='Other'].index, inplace=True)
df_test.drop(df_test[df_test['gender']=='Other'].index, inplace=True)

In [ ]:
df_train = pd.get_dummies(df_train, columns=['company_size'], prefix='company_size')
df_test = pd.get_dummies(df_test, columns=['company_size'], prefix='company_size')

In [ ]:
df_train['training_hours'] = (df_train['training_hours'] - df_train['training_hours'].mean())/df_train['training_hours'].std()
df_test['training_hours'] = (df_test['training_hours'] - df_test['training_hours'].mean())/df_test['training_hours'].std()

In [ ]:
a = list(['city_development_index','gender','relevent_experience','experience','last_new_job','training_hours','target'])
heatmap_data = df_train[a]

colormap = plt.cm.RdBu
plt.figure(figsize=(18, 14))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(heatmap_data.astype(float).corr(), linewidths=0.1, vmax=1.0,
           square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 16})

del heatmap_data

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

gb = GradientBoostingClassifier(random_state=42)

y_target = df_train['target']
X_data = df_train.drop(['target'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.3, random_state=42)

param_grid = [{'n_estimators': range(1, 500, 50), 'max_features': range(1, 10),
              'max_depth': range(3, 5), 'learning_rate': np.linspace(0.1, 0.1, 10)}]
gs = GridSearchCV(estimator=gb, param_grid=param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs.fit(X_train, y_train)

In [ ]:
gb_pred = gs.predict(X_test)

In [ ]:
gb_pred = gs.predict(X_test)
from sklearn.metrics import accuracy_score
gb_acc = accuracy_score(y_test, gb_pred)
gb_acc

In [ ]:
x_submit = df_submit.drop(['target'], axis=1, inplace=False)
prediction = gs.predict(X_test)
prediction_df = pd.Series(prediction)
type(prediction_df)

In [ ]:
df_submit['target'] = prediction_df
df_submit.describe()

In [ ]:
df_submit.head(10)

In [ ]:
df_submit.to_csv('my_submission.csv', index=False)